In [1]:
import pandas as pd
import statistics

In [2]:
Data2010 = pd.read_excel('OViN2010_Databestand.xlsx')
Data2011 = pd.read_excel('OViN2011_Databestand.xlsx')
Data2012 = pd.read_excel('OViN2012_Databestand.xlsx')
Data2013 = pd.read_excel('OViN2013_Databestand.xlsx')
Data2014 = pd.read_excel('OViN2014_Databestand.xlsx')
Data2015 = pd.read_excel('OViN2015_Databestand.xlsx')
Data2016 = pd.read_excel('OViN2016_Databestand.xlsx')
Data2017 = pd.read_excel('OViN2017_Databestand.xlsx')

In [3]:
df = pd.concat([Data2010, Data2011, Data2012, Data2013, Data2014, Data2015, Data2016, Data2017])

C:\Users\sjoer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [4]:
# Select relevant columns
df = df[['OPID', 'Weekdag', 'VertUur', 'AankUur', 'VertMin', 'AankMin', 'BVertUur', 'BVertMin', 'BAankUur', 'BAankMin', 'RVertUur', 'RVertMin', 'RAankUur', 'RAankMin', 'Rvm', 'Doel', 'Reisduur', 'ReisduurBL', 'RReisduur', 'RReisduurBL', 'Toer']]

#Select only movements by car
df = df[df['Rvm'] == 6]
df = df[df['Toer'] == 0]

# fill nan-values of a trip with the values of the whole movement
df['RVertUur'] = df['RVertUur'].fillna(df['BVertUur'])
df['RVertMin'] = df['RVertMin'].fillna(df['BVertMin'])
df['RAankUur'] = df['RAankUur'].fillna(df['BAankUur'])
df['RAankMin'] = df['RAankMin'].fillna(df['BAankMin'])

df['RVertUur'] = df['RVertUur'].fillna(df['VertUur'])
df['RVertMin'] = df['RVertMin'].fillna(df['VertMin'])
df['RAankUur'] = df['RAankUur'].fillna(df['AankUur'])
df['RAankMin'] = df['RAankMin'].fillna(df['AankMin'])

df['RReisduur'] = df['RReisduur'].fillna(df['Reisduur'])
df['RReisduurBL'] = df['RReisduurBL'].fillna(df['ReisduurBL'])

# Drop nan-values
df = df.dropna(subset=['RReisduur', 'RVertUur', 'RAankUur'])
df = df.reset_index(drop = True)

df['Total_Reisduur'] = df['RReisduur'] + df['RReisduurBL']

In [5]:
# Combine the two goals bringing people and bringing goods
df.loc[df['Doel'] == 6, 'Doel'] = 5

In [6]:
# Make a dataframe that can be filled with the data
triplength = pd.DataFrame(0, index = list(range(1, 4)) + [5] + list(range(7, 15)), columns = ['Average', 'Standard Deviation'])

In [7]:
# Add the averages to the dataframe triplength
triplength['Average'] = df.groupby(['Doel']).mean()['Total_Reisduur']

In [8]:
# Add the standard deviations to the dataframe triplength
for doel in triplength.index:
    df1 = df[df['Doel'] == doel]
    triplength.loc[doel, 'Standard Deviation'] = statistics.stdev(df1['Total_Reisduur'])

In [9]:
triplength

,Average,Standard Deviation
1,24.345453,34.589046
2,28.940259,33.062731
3,44.017846,56.973958
5,17.013796,22.237043
7,25.885237,29.238809
8,14.573254,18.855424
9,26.885953,33.025130
10,37.090179,54.956612
11,19.303639,25.016012
12,28.267114,40.579902


In [10]:
# Save the triplength
triplength.to_csv('Triplength.csv')

In [2]:
triplength = pd.read_csv('Triplength.csv').set_index('Unnamed: 0')
del triplength.index.name

In [4]:
# Set the goal as index for understandability of the dataframe and print the dataframe
triplength.index = ['Home', 'Work', 'Work_Visit', 'Bringing', 'Education', 'Shop', 'Visit', 'Wandering', 'Hobby', 'Leisure', 'Personal_Care', 'Other']
triplength.round(2)

,Average,Standard Deviation
Home,24.35,34.59
Work,28.94,33.06
Work_Visit,44.02,56.97
Bringing,17.01,22.24
Education,25.89,29.24
Shop,14.57,18.86
Visit,26.89,33.03
Wandering,37.09,54.96
Hobby,19.30,25.02
Leisure,28.27,40.58
